### 배경

Open-Domain Question Answering (ODQA) 문제의 해결을 위한 연구는 모델 파라미터와 별개로 외부 corpus에 접근할 수 있는지 여부에 따라 Open-Book 모델과 Closed-Book 모델로 구분될 수 있습니다(Roberts et al., 2020). Open-Book 모델 중 Lewis et al.(2020)의 RAG 모델은 질문에 답변하기 위하여 인공신경망이 참조할 수 있는 외부 corpus를 검색하는 retrieval과, 찾아낸 corpus를 참조하여 답변을 생성하는 generator을 결합하여 주목할만한 결과를 얻었습니다. RAG는 retrieval과 generator로 각각 DPR(Karpukhin et al., 2020)과 BART(Lewis et al., 2020)을 사용하였습니다.

본 주피터 노트북은 HuggingFace NLP course를 공부한 내용을 정리하기 위해 retriever와 generator를 결합한 RAG의 방식을 차용하여 Question Answering을 연습해본 기록입니다. HuggingFace 웹페이지에서 제공하는 코드들을 응용하여 작성해보았습니다. 다만 기본적인 형태를 연습하는 과정에서 BERT의 embedding space를 사용한 RAG 모델과는 달리 retriever로 MPNet Embedding Space를 활용하여 Semantic Search를 수행하였고, generator로는 HuggingFace Transformer에서 기본적으로 제공하는 text2text-generation pipeline 및 eli5 dataset에 대하여 fine-tuning된 BART 모델을 사용하였습니다. 시간관계 상, 그리고 gpu 사양의 한계로 본 노트북에서 따로 fine-tuning을 수행하지는 않았습니다. 추후에는 특정 분야에 대한 전문적인 corpus를 대상으로 finetuning을 진행해보고자 합니다.

In [ ]:
# 아래와 같은 라이브러리 설치가 필요합니다. faiss-gpu의 경우 Linux 기준 pip를 통해 설치할 수 있으나 저는 윈도우에서 작업하여 conda를 사용하였습니다.
# 윈도우 기준 conda를 통해 faiss-gpu를 설치한 경우는 커널을 재시작해주어야 정상작동하는 모습을 보였습니다. 

# !pip pytorch datasets evaluate transformers[sentencepiece]
# conda install -c conda-forge faiss-gpu

In [1]:
# # requirements

# pytorch
# datasets
# evaluate
# transformers[sentencepiece]

# # conda install -c conda-forge faiss-gpu

### **Load Dataset**

Question과 Answer 그리고 Context가 모두 주어진 데이터셋을 사용하고자 하였고, SQuAD를 선택하였습니다.\
노트북의 하단에서는 eli5 데이터셋을 사용하여 동일한 작업을 수행합니다. 

In [1]:
# SQuAD 데이터셋을 불러옵니다.
from datasets import load_dataset

raw_dataset = load_dataset("squad", split="train")
raw_dataset

c:\Users\user\Documents\23_05_RAG\RAG\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset squad (C:/Users/user/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

In [14]:
# 랜덤으로 N개의 샘플을 선택합니다. 본 노트북에서는 따로 Training을 수행하지는 않으며
# N개의 sample만을 대상으로 semantic search와 generation task를 수행하였습니다. 
N = 1000
sample_dataset = raw_dataset.shuffle(seed=42).select(range(N)).select_columns(['question', 'answers', 'context'])
sample_dataset

Loading cached shuffled indices for dataset at C:\Users\user\.cache\huggingface\datasets\squad\plain_text\1.0.0\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453\cache-10de9997c4b83f65.arrow


Dataset({
    features: ['context', 'question', 'answers'],
    num_rows: 1000
})

In [5]:
sample_dataset[0]

{'context': 'The Pew Forum on Religion & Public Life ranks Egypt as the fifth worst country in the world for religious freedom. The United States Commission on International Religious Freedom, a bipartisan independent agency of the US government, has placed Egypt on its watch list of countries that require close monitoring due to the nature and extent of violations of religious freedom engaged in or tolerated by the government. According to a 2010 Pew Global Attitudes survey, 84% of Egyptians polled supported the death penalty for those who leave Islam; 77% supported whippings and cutting off of hands for theft and robbery; and 82% support stoning a person who commits adultery.',
 'question': 'What percentage of Egyptians polled support death penalty for those leaving Islam?',
 'answers': {'text': ['84%'], 'answer_start': [468]}}

### **Retrieval** (Semantic Search with FAISS)

In [3]:
#MPNet을 불러와 tokenizer와 embedding 모델로 사용합니다.
from transformers import AutoTokenizer, AutoModel

mpnet_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
mpnet_tokenizer = AutoTokenizer.from_pretrained(mpnet_ckpt)
retriever = AutoModel.from_pretrained(mpnet_ckpt)

In [4]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)
retriever.to(device)

cuda


MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0-11): 12 x MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_

In [5]:
# gpu 메모리 관리를 위한 함수
import gc

def clean():
    gc.collect()
    torch.cuda.empty_cache()

In [6]:
# 불러온 모델을 활용해 input에 대한 embedding을 얻는 함수를 작성합니다.
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

def get_embeddings(text_list, model):
    encoded_input = mpnet_tokenizer(
        text_list, 
        padding=True, 
        truncation=True, 
        return_overflowing_tokens=True,
        return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    result = cls_pooling(model_output).detach().cpu().numpy()[0]
    
    del encoded_input, model_output
    clean()
    
    return result

In [10]:
# embedding shape preview
embedding = get_embeddings(sample_dataset["context"][0], retriever)
print(embedding.shape)
del embedding

(768,)


In [11]:
# map 함수를 사용해 dataset의 모든 context에 대한 embedding을 얻습니다. 이후 faiss index를 추가합니다.
embeddings_dataset = sample_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["context"], retriever)}
)
embeddings_dataset.add_faiss_index(column="embeddings")

Loading cached processed dataset at C:\Users\user\.cache\huggingface\datasets\squad\plain_text\1.0.0\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453\cache-041716c3067efe4e.arrow
100%|██████████| 1/1 [00:00<00:00, 167.12it/s]


Dataset({
    features: ['context', 'question', 'answers', 'embeddings'],
    num_rows: 1000
})

In [15]:
# 저장과 불러오기를 위한 구간
import pickle

with open(file='embeddings_dataset_squad_{}.pickle'.format(N), mode='wb') as f:
    pickle.dump(embeddings_dataset, f)
    
# with open(file='embeddings_dataset_squad_{}.pickle'.format(N), mode='rb') as f:
#     embeddings_dataset=pickle.load(f)

In [16]:
# N개의 데이터에 대하여 특정 index의 question과 answer를 불러옵니다. 또한 question에 대한 embedding을 얻습니다. 
idx = 125

question = sample_dataset['question'][idx]
question_embedding = get_embeddings([question], retriever)
question_embedding.shape
print('Q: ', question)

answer = sample_dataset['answers'][idx]['text'][0]
print('A: ', answer)

Q:  The energy used for metabolism of the brain in humans is what percentage?
A:  20–25%


In [17]:
# 위에서 얻은 question embedding이 context embedding과 갖는 유사도를 계산하여 가장 유사한 k개의 sample을 얻습니다.
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=10
)

In [18]:
# pandas로 시각화
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=True, inplace=True)

samples_df

,context,question,answers,embeddings,scores
0,Brain tissue consumes a large amount of energy...,The energy used for metabolism of the brain in...,"{'text': ['20–25%'], 'answer_start': [559]}","[0.32254528999328613, -0.4942903518676758, -0....",29.692329
1,Some types of energy are a varying mix of both...,What is dependent upon electrical potential en...,"{'text': ['Elastic energy in materials'], 'ans...","[0.1286253184080124, -0.27638325095176697, -0....",45.315144
2,Early recommendations for the quantity of wate...,How much water should be taken in for each cal...,"{'text': ['1 milliliter'], 'answer_start': [469]}","[0.23622414469718933, -0.18039996922016144, -0...",47.218964
3,Bacteria are further divided into lithotrophs ...,What do respiratory organisms use as electron ...,"{'text': ['chemical compounds'], 'answer_start...","[0.10745195299386978, -0.362464964389801, -0.1...",48.950485
4,Efficiency of a transmitting antenna is the ra...,What can cause that reaction?,"{'text': ['transmitter'], 'answer_start': [494]}","[-0.09236371517181396, -0.733789324760437, -0....",48.979622
5,Only a few contemporary societies are classifi...,"Besides agriculture, how do gatherers add to ...","{'text': ['keeping animals'], 'answer_start': ...","[-0.30304154753685, 0.40761739015579224, -0.19...",49.569466
6,Dietary fiber is a carbohydrate that is incomp...,What is an example of a gastrointestinal probl...,"{'text': ['constipation'], 'answer_start': [771]}","[-0.07349792867898941, 0.10766194015741348, -0...",49.936504
7,"The judges continue in paragraph 12, ""The dete...",The number of people targeted in a genocide sh...,"{'text': ['absolute terms'], 'answer_start': [...","[0.22658081352710724, -0.41311776638031006, -0...",50.440056
8,Spanish is currently the most widely taught no...,What other languages are popular among America...,"{'text': ['French (14.4%), German (7.1%), Ital...","[-0.18415644764900208, -0.2967764437198639, -0...",50.696846
9,Bacterial growth follows four phases. When a p...,What is called the third statge of growth of b...,"{'text': ['stationary phase'], 'answer_start':...","[0.23717251420021057, -0.4525332450866699, -0....",50.884193


In [19]:
# 찾아낸 k개의 context들을 유사도가 높은 것부터 결합하여 list로 반환합니다.
samples_df['context'].to_list()

["Brain tissue consumes a large amount of energy in proportion to its volume, so large brains place severe metabolic demands on animals. The need to limit body weight in order, for example, to fly, has apparently led to selection for a reduction of brain size in some species, such as bats. Most of the brain's energy consumption goes into sustaining the electric charge (membrane potential) of neurons. Most vertebrate species devote between 2% and 8% of basal metabolism to the brain. In primates, however, the percentage is much higher—in humans it rises to 20–25%. The energy consumption of the brain does not vary greatly over time, but active regions of the cerebral cortex consume somewhat more energy than inactive regions; this forms the basis for the functional brain imaging methods PET, fMRI, and NIRS. The brain typically gets most of its energy from oxygen-dependent metabolism of glucose (i.e., blood sugar), but ketones provide a major alternative source, together with contributions 

In [21]:
# 위의 list 원소들을 문자열로 변환하고 결합합니다. 이것이 질문과 함께 제공되는 우리의 external corpus (context)가 됩니다. 
context = str()
for sentence in samples_df['context'].to_list():
    context += sentence

### **Generator**

HuggingFace Transformer에서 제공하는 "text2text-generation" pipeline을 generator로 사용하였습니다. 

In [26]:
from transformers import pipeline

text2text_generator = pipeline("text2text-generation")

No model was supplied, defaulted to t5-base and revision 686f1db (https://huggingface.co/t5-base).
Using a pipeline without specifying a model name and revision in production is not recommended.
c:\Users\user\Documents\23_05_RAG\RAG\lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [27]:
# 차례대로 주어진 질문, 모델이 뱉어낸 답변 및 정답입니다.
print(question)
output = text2text_generator("question: {} context: {}".format(question, context))
print(output[0]['generated_text'])
print(answer)

Token indices sequence length is longer than the specified maximum sequence length for this model (2143 > 512). Running this sequence through the model will result in indexing errors


The energy used for metabolism of the brain in humans is what percentage?
20–25%
20–25%


In [29]:
# 질문에 대해 찾아낸 context와 관련하여 다른 질문을 던져보았습니다. 적절히 대답하는 모습을 보여줍니다.
question2 = 'how much energy does fly use for metabolism of the brain?'
output = text2text_generator("question: {} context: {}".format(question2, context))
print(output[0]['generated_text'])

2% to 8%


### **eli5 dataset with BART Generator**

이번에는 eli5 데이터셋을 사용하여 같은 방식을 적용하되, BART를 Generator로 사용하고자합니다.\
https://yjernite.github.io/lfqa.html#dense_use의 코드를 참조하였으며,\
사용한 BART 모델 역시 해당 링크에서 eli5 데이터셋에 대하여 fine-tuning을 진행한 모델입니다.

In [7]:
# eli5 데이터셋을 불러옵니다. 과학을 주제로 하는 'asks' split을 사용하고자합니다. 
raw_dataset = load_dataset("eli5", split="train_asks")

Found cached dataset eli5 (C:/Users/user/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa)


In [8]:
# 마찬가지로 1000개의 sample을 뽑아 사용합니다. title과 answer를 각각 question과 context로 변경하여 사용할 것입니다. 
N = 1000
sample_dataset = raw_dataset.shuffle(seed=42).select(range(N)).select_columns(['title', 'answers'])
sample_dataset = sample_dataset.rename_column('title', 'question')
sample_dataset = sample_dataset.rename_column('answers', 'context')
sample_dataset

Loading cached shuffled indices for dataset at C:\Users\user\.cache\huggingface\datasets\eli5\LFQA_reddit\1.0.0\17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa\cache-5c2bb45099cb5c79.arrow


Dataset({
    features: ['question', 'context'],
    num_rows: 1000
})

In [9]:
# context에 답변의 id가 포함되어있습니다. 이를 무시하고 답변 내용인 text만 사용하고자합니다. 
sample_dataset.set_format("pandas")
sample_df = sample_dataset[:]
sample_df

,question,context
0,Are there any equivalents to logical gates in ...,"{'a_id': ['c7ni94z', 'c7nil06'], 'text': ['Neu..."
1,"If you eat the same thing everyday, does your ...","{'a_id': ['cerowjh'], 'text': ['There is a ver..."
2,"Statistically speaking, how far away should th...","{'a_id': ['ccgb84o'], 'text': ['The [nearest b..."
3,Geologists & Geology enthusiasts: What could...,"{'a_id': ['cawg229'], 'text': ['Before I say a..."
4,How can scattering of blue light simultaneousl...,"{'a_id': ['cskd9um', 'cskf4c7'], 'text': ['Day..."
...,...,...
995,Where do autistic people derive their of moral...,"{'a_id': ['c6qj8wu', 'c6qn82j', 'c6qmj1r', 'c6..."
996,"How can we know things like how many people ""d...","{'a_id': ['dzx1btj'], 'text': ['We can only kn..."
997,What would happen to an eye completely bereft ...,"{'a_id': ['cp7g3f4'], 'text': ['In case anyone..."
998,Big Bang and the accelerating universe?,"{'a_id': ['ckbywpd', 'ckbzu38'], 'text': ['In ..."


In [10]:
# apply와 lambda 함수를 사용해 답변의 text만을 context로 사용합니다. 
sample_df['context'] = sample_df['context'].apply(lambda x: x['text'])

# 하나의 질문에 대해 여러 답변이 달린 경우를 explode 함수를 사용해 분리해줍니다.  
sample_df = sample_df.explode("context", ignore_index=True)
sample_df.head()

,question,context
0,Are there any equivalents to logical gates in ...,Neurons function similar way as logic gates i...
1,Are there any equivalents to logical gates in ...,Heh. We wish we had found such a thing.\n\nTh...
2,"If you eat the same thing everyday, does your ...",There is a very substantial variation among th...
3,"Statistically speaking, how far away should th...",The [nearest brown dwarf](_URL_0_) -- a pair o...
4,Geologists & Geology enthusiasts: What could...,"Before I say anything stupid, is there any cha..."


In [11]:
# pandas 데이터프레임을 다시 dataset으로 변경합니다. 
from datasets import Dataset

sample_dataset = Dataset.from_pandas(sample_df)

In [ ]:
# map 함수를 사용해 dataset의 모든 context에 대한 embedding을 얻습니다. 이후 faiss index를 추가합니다.
embeddings_dataset = sample_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["context"], retriever)}
)
embeddings_dataset.add_faiss_index(column="embeddings")

# 저장과 불러오기를 위한 구간
with open(file='embeddings_dataset_eli5_{}.pickle'.format(N), mode='wb') as f:
    pickle.dump(embeddings_dataset, f)

# with open(file='embeddings_dataset_eli5_{}.pickle'.format(N), mode='rb') as f:
#     embeddings_dataset=pickle.load(f)

In [34]:
# N개의 데이터에 대하여 특정 index의 question과 answer를 불러옵니다. 또한 question에 대한 embedding을 얻습니다. 
idx = 0

question = sample_dataset['question'][idx]
question_embedding = get_embeddings([question], retriever)

print('Q: ', question)

Q:  Are there any equivalents to logical gates in the nervous system?


In [16]:
# 위에서 얻은 question embedding이 context embedding과 갖는 유사도를 계산하여 가장 유사한 k개의 sample을 얻습니다.
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=10
)

In [17]:
# pandas로 시각화
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=True, inplace=True)

samples_df

,question,context,embeddings,scores
0,Are there any equivalents to logical gates in ...,Heh. We wish we had found such a thing.\n\nTh...,"[-0.06319624185562134, -0.027759850025177002, ...",24.671970
1,"Is there any legitimacy to ""neuro training"" si...",[Here](_URL_0_) you go. This should answer you...,"[-0.1310725212097168, -0.41311880946159363, -0...",37.346756
2,why is the bond length and angle of phosphorus...,This sounds a lot like a homework question to ...,"[-0.228667214512825, -0.20940203964710236, -0....",39.054783
3,If you could get to the centre of a black hole...,[Here](_URL_0_) is RobotRollCall's comment on ...,"[-0.117356076836586, -0.3746531009674072, -0.3...",39.055691
4,What everyday problems could possibly be solve...,This is an incredibly vague question. If you h...,"[-0.29924556612968445, -0.27476516366004944, -...",39.088982
5,How can scattering of blue light simultaneousl...,Walter Lewin demonstrates it (and much more!) ...,"[-0.31137293577194214, -0.6795521378517151, -0...",39.189892
6,Where do autistic people derive their of moral...,This might help a bit. \n\nThe Neurobiology of...,"[-0.19906999170780182, -0.0950225293636322, -0...",39.694599
7,Are there any equivalents to logical gates in ...,Neurons function similar way as logic gates i...,"[0.4118035137653351, -0.5458545684814453, 0.08...",39.894703
8,Is this Russian methane release as ominous as ...,There is already [a discussion](_URL_1_) on th...,"[-0.02792995236814022, -0.2829281985759735, -0...",40.497002
9,Do we perceive time to pass at the same rate i...,Here are a couple of previous threads on the t...,"[-0.16528888046741486, -0.3233572244644165, -0...",41.208870


In [18]:
# 찾아낸 k개의 context들을 유사도가 높은 것부터 결합하여 str으로 반환합니다. 
context = str()
for sentence in samples_df['context'].to_list():
    context += sentence

In [19]:
# eli5 dataset에 대하여 pre-trained된 모델을 불러옵니다. 
# https://yjernite.github.io/lfqa.html#dense_use

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

ckpt = "yjernite/bart_eli5"
bart_tokenizer = AutoTokenizer.from_pretrained(ckpt)
bart_model = AutoModelForSeq2SeqLM.from_pretrained(ckpt).to(device)

In [55]:
# 질문에 대한 토큰 id와 mask를 가져옵니다. 본 예시에서는 하나의 input을 입력하여 padding을 사용하지 않으므로 attention_mask는 모두 1이 됩니다. 
q_toks = bart_tokenizer.batch_encode_plus([question], truncation=True)
q_ids, q_mask = (
    torch.LongTensor(q_toks["input_ids"]).to(device),
    torch.LongTensor(q_toks["attention_mask"]).to(device),
)

In [49]:
# 토큰 id, mask의 모습 
q_ids, q_mask

(tensor([[    0, 13755,    89,   143, 26699,     7, 16437, 14213,    11,     5,
           7464,   467,   116,     2]], device='cuda:0'),
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0'))

In [50]:
# model input을 생성합니다. 
model_inputs = {
    "input_ids": q_ids,
    "attention_mask": q_mask
}
model_inputs

{'input_ids': tensor([[    0, 13755,    89,   143, 26699,     7, 16437, 14213,    11,     5,
           7464,   467,   116,     2]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [51]:
# 모델 입력을 통해 답변 ids를 생성합니다. 
generated_ids = bart_model.generate(
        input_ids=model_inputs["input_ids"],
        attention_mask=model_inputs["attention_mask"],
        min_length=60,
        max_length=120,
        do_sample=False,
        early_stopping=True,
        num_beams=1,
        temperature=1.0,
        top_k=None,
        top_p=None,
        eos_token_id=bart_tokenizer.eos_token_id,
        no_repeat_ngram_size=3,
        num_return_sequences=1,
        decoder_start_token_id=bart_tokenizer.bos_token_id,
)

In [52]:
generated_ids

tensor([[    0,    20,  7464,   467,    16,    10,   182,  2632,   467,     4,
            85,    18,    45,    95,    10,  6900,     9, 21737,     8, 37409,
             4,    85,    34,    10,   319,     9,   430,  1667,     4,    20,
          7464,  1743,    32,   156,    62,     9,    10,   319,    55,    87,
            95,    10,   367, 21737,     8,    10,   367, 37409,     4,    20,
         17358,    32,   156,     9,    10,  6900,    55,   383,    87,    95,
         21737,     8,     5, 17358,    32,    67,   156,    62,    30,    10,
          6900,   114,   383,   101, 14357,     8,  3805,  1790,     4,    20,
         10387,    16,   156,    62,  2260,     9,    10,  1086,  6900,     9,
           430,   383,     4,    20,  2900,    16,   156,     9,  3739,     9,
           430,  6134,     9,   383,     4,    85,    16,   156,  2260,     9,
          3739,     8,  3739,     9,   383,    14,    32,   156,    66,     2]],
       device='cuda:0')

In [53]:
# 생성된 id를 decoding하면 다음과 같은 결과를 볼 수 있습니다. 
bart_tokenizer.decode(generated_ids[0], skip_special_tokens=True)

" The nervous system is a very complex system. It's not just a bunch of switches and levers. It has a lot of different parts. The nervous systems are made up of a lot more than just a few switches and a few levers. The nerves are made of a bunch more things than just switches and the nerves are also made up by a bunch if things like muscles and tendons. The nerve is made up mostly of a whole bunch of different things. The brain is made of lots of different kinds of things. It is made mostly of lots and lots of things that are made out"

In [54]:
# context와 비교하여 보면, 모델이 적절한 답변을 생성해내는 것을 확인할 수 있습니다.  
print('[question]:', question, '\n')
print('[context]:', context)

[question]: Are there any equivalents to logical gates in the nervous system? 

[context]: Heh.  We wish we had found such a thing.

This is a contentious issue, actually.  [Computationalists](_URL_3_) (of which I am one) would say, yes, there have to be... somewhere.  But we don't exactly know where.  [Neural network](_URL_0_) theorists, meanwhile, say that logical gates and computational frameworks are unnecessary; you just need to figure out the strength of connections between different neurons.  They're in love with [LTP](_URL_1_), which they claim backs them up.  And, well, we have actual evidence for LTP.  However, computationalists fire back and say, no, connection strength isn't enough, because then you can't have the brain acting like a Turing machine, which would ultimately be nice for a variety of reasons.  Not in the least, if we're trying to effect a reasonable response to the environment, it would be nice to reconstruct inputs from outputs.  Plus, representations are just